# Nigeria 1983 - 2016
By Cascade Tuholske, cascade dot tuholske 1 at montana dot edu 

Notebook to make a Figure 1 of the average annual WBGTmax for 1983 and 2016 and Difference for Nigeria 

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show

## Make a Dif Raster

In [ ]:
# open raster as arrays
ob2016 = rasterio.open(os.path.join('PATH/TO/DATA/GHE-wbgtmax-avg-2016.tif')).read(1)
ob1983 = rasterio.open(os.path.join('PATH/TO/DATA/GHE-wbgtmax-avg-1983.tif')).read(1)

In [ ]:
# Check the data
plt.imshow(np.ma.masked_equal(ob1983, -9999))
plt.colorbar()

In [ ]:
# Make a masked difference raster (note one Pixel in Bangladesh in land in 1983 but not in 2016).  
dif = ob2016 - ob1983
ocean = np.where(ob1983 == -9999, -9999, 0)
dif = dif + ocean 
print(dif.min())
dif = np.where(dif < -9999, -9999, dif) # set the Bangladesh pixel to ocean
print(dif.min())
dif_mask = np.ma.masked_equal(dif, -9999) # mask the array for plotting

In [ ]:
# Check dif_mask
plt.imshow(dif_mask)
plt.colorbar()

In [ ]:
# change the data type of dif
dif = dif.astype(np.float32)

In [ ]:
# write it out 
meta = rasterio.open(os.path.join('PATH/TO/DATA/GHE-wbgtmax-avg-2016.tif')).meta

fn_out = os.path.join('PATH/TO/DATA/GHE-wbgtmax-avg-8316.tif')
with rasterio.open(fn_out, 'w', **meta) as out:
    out.write_band(1, dif)

## Plot

In [ ]:
# open shapefile from Natural Earth: 
# https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-0-countries/
shps = gpd.read_file(os.path.join('PATH/TO/DATA/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp'))
shps.crs

In [ ]:
# Open the rst as rasterio objects
ob2016 = rasterio.open(os.path.join('PATH/TO/DATA/GHE-wbgtmax-avg-2016.tif'))
ob1983 = rasterio.open(os.path.join('PATH/TO/DATA/GHE-wbgtmax-avg-1983.tif'))
dif = rasterio.open(os.path.join('PATH/TO/DATA/GHE-wbgtmax-avg-8316.tif'))

In [ ]:
# set up

fig, axs = plt.subplots(nrows=1,ncols=3, figsize=(25,15))

ws = 0.05
fig.subplots_adjust(wspace=ws)

# 1983
im1 = show(ob1983, ax = axs[0], vmin = 15, vmax = 30, cmap = 'hot_r')
shps.boundary.plot(ax = axs[0], color = 'gray')

# 2016 
im2 = show(ob2016, ax = axs[1], vmin = 15, vmax = 30, cmap = 'hot_r')
shps.boundary.plot(ax = axs[1], color = 'gray')

# dif 
im3 = show(dif, ax = axs[2], vmin = 0, vmax = 1.75, cmap = 'plasma_r')
shps.boundary.plot(ax = axs[2], color = 'gray')

# color bar
cb1 = fig.colorbar(mappable = im1.get_images()[0], ax = axs[0], shrink = 0.35)
cb2 = fig.colorbar(mappable = im2.get_images()[0], ax = axs[1], shrink = 0.35)
cb3 = fig.colorbar(mappable = im3.get_images()[0], ax = axs[2], shrink = 0.35)

cb1.set_label('°C', fontsize=12, rotation = 0, labelpad=10)
cb2.set_label('°C', fontsize=12, rotation = 0, labelpad=10)
cb3.set_label('°C', fontsize=12, rotation = 0, labelpad=10)

# limits
xmin = 2.5
xmax = 15
ymin = 4
ymax = 14

axs[0].set_xlim(xmin, xmax);
axs[0].set_ylim(ymin, ymax);
axs[1].set_xlim(xmin, xmax);
axs[1].set_ylim(ymin, ymax);
axs[2].set_xlim(xmin, xmax);
axs[2].set_ylim(ymin, ymax);

# remove the x and y ticks
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

# Titles
axs[0].set_title('1983', fontsize = 17);
axs[1].set_title('2016', fontsize = 17)
axs[2].set_title('Increase 1983 - 2016', fontsize = 16)

#fig.subplots_adjust(top = 1.47)
#fig.suptitle('Average Daily Maximum WBGT for Nigeria', fontsize = 20);

# Add panel letters (a), (b), (c)
axs[0].text(-0.1, 1.05, '(a)', transform=axs[0].transAxes, fontsize=15, va='top'); #  fontweight='bold',
axs[1].text(-0.1, 1.05, '(b)', transform=axs[1].transAxes, fontsize=15, va='top');
axs[2].text(-0.1, 1.05, '(c)', transform=axs[2].transAxes, fontsize=15, va='top');

# save it out
plt.savefig(os.path.join('PATH/TO/DATA/wbgtmax-avg1983-2016-ng.pdf'), dpi = 300, bbox_inches='tight')